<a href="https://colab.research.google.com/github/ymdysk/chatrwkv-notebook/blob/main/ChatRWKV_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatRWKVをGoogle Colabで実行するnotebook
## 動作概要
- Google Driveをマウントし、srcフォルダにChatRWKVのGitリポジトリをcloneします。
- ChatRWKV/v2/ フォルダに移動してモデルをダウンロードします。
- モデルに関する変数はフォームで設定し、chat.pyの内容を一部置換してchat-notebook.pyに保存してモデルをロードします。
- 文章生成に関する変数を設定してチャットを実行します。

## Tips
- GPUを使いたい場合はGoogle Colabの「ランタイム」メニュー→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」で「GPU」を選択して保存します。
- GPUが使えない場合は strategy = 'cpu fp32', RWKV_CUDA_ON = 0 を設定すると動作しますが、生成速度は遅くなります。
- 使用するモデルは、約3GBの1B(10億)パラメータのモデルか約6GBの3B(30億)パラメータのモデルをMODEL_URLで選択するか、任意のURLを入力します。パラメータ数のより多いモデルを使うとより賢い応答ができますが、ファイル容量も大きくなり、VRAMやRAM等の要求スペックも上がります。
- Google Colaboratoryの無料版では1B,3B迄のモデルは実行できるようです。
- Googleの無料ストレージは15GBですが、Google Driveにモデルを保存すると逼迫しやすいです。
- 2023年4月21日時点では、Hugging Faceで公開されている各モデルの日本語の割合は0～10%程度です。日本語の応答をより自然にするためには、日本語の割合の高いデータでプレトレーニングしたり、ファインチューニングしたりする必要があると思われます。

## License
- このnotebook  
  Copyright 2023 Yosuke Yamada  
  Licensed under the Apache License, Version 2.0  
  http://www.apache.org/licenses/LICENSE-2.0
- notebookから使用/ダウンロードするソフトウェアやモデルのライセンスは個別に確認してください。



# 環境の確認

In [ ]:
# NVIDIAのシステム管理インターフェイスでCUDA環境の状況を確認(GPU用。CPUを使う場合は実行不要)
!nvidia-smi

Sun Apr 23 05:27:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Cuda compilerのバージョンを確認(GPU用。CPUを使う場合は実行不要)
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# 環境設定

In [ ]:
# デフォルトのtorchでは高速化のためRWKV_CUDA_ON = 1を設定した場合に、CUDAカーネルのビルドに失敗することがあるので入れなおす(初回のみ)
!pip uninstall -y torch
!pip install torch==2.0.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

Found existing installation: torch 2.0.0+cu118
Uninstalling torch-2.0.0+cu118:
  Successfully uninstalled torch-2.0.0+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 772.7 kB/s eta 0:00:00


In [ ]:
# PytorchでCUDAが使えるか確認する。使える場合、それぞれTrue, デバイス番号が返される。失敗する場合は上のセルを再実行する。CPUを使う場合(GPUを使わない場合)は実行不要。
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())

True
0


In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')
# ソースコード配置先フォルダがなければ作成してcdする
import os
os.makedirs("/content/drive/My Drive/src", exist_ok=True)
%cd '/content/drive/My Drive/src'

Mounted at /content/drive
/content/drive/My Drive/src


In [ ]:
# ChatRWKVのソースコードをgit cloneで取得(初回のみ)
!git clone https://github.com/BlinkDL/ChatRWKV

Cloning into 'ChatRWKV'...
remote: Enumerating objects: 1340, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 1340 (delta 88), reused 162 (delta 52), pack-reused 1124
Receiving objects: 100% (1340/1340), 26.97 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (723/723), done.


In [ ]:
# ChatRWKV/v2フォルダに移動
%cd 'ChatRWKV/v2'

/content/drive/MyDrive/src/ChatRWKV/v2


In [ ]:
# rwkv, ninjaパッケージをpipでインストール
!pip install rwkv ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.1 MB/s eta 0:00:00


# モデルの準備

In [ ]:
# モデルの設定・ダウンロード
# Raven  https://huggingface.co/BlinkDL/rwkv-4-raven/tree/main
# その他 https://huggingface.co/BlinkDL
MODEL_URL = 'https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-3B-v8-EngAndMore-20230408-ctx4096.pth' #@param ['https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-1B5-v10-Eng99%25-Other1%25-20230418-ctx4096.pth', 'https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-3B-v8-EngAndMore-20230408-ctx4096.pth"]  {allow-input: true}
!curl -OLC - $MODEL_URL
# MODEL_NAME は MODEL_URL の最後の「/」記号以降の文字列(ファイル名)から、拡張子の「.pth」を除いたもの
MODEL_NAME = MODEL_URL[MODEL_URL.rfind('/') + 1:].rstrip('.pth')

In [ ]:
# RWKVモデルの動作に関する設定
strategy = 'cuda fp16' #@param ['cpu fp32', 'cuda fp16', 'cuda:0 fp16 -> cuda:1 fp16', 'cuda fp16i8 *10 -> cuda fp16', 'cuda fp16i8', 'cuda fp16i8 -> cpu fp32 *10', 'cuda fp16i8 *10+'] {allow-input: true}
# 'cpu fp32' # CUDAが使えずCPUを使う場合
# 'cuda fp16' # CUDAが使える場合のデフォルト値
# 'cuda:0 fp16 -> cuda:1 fp16' # 2枚のGPUが使える場合
# 'cuda fp16i8 *10 -> cuda fp16' # 最初は10層cuda int8量子化、残りはcuda fp16i8
# 'cuda fp16i8' # 全層cuda int8量子化
# 'cuda fp16i8 -> cpu fp32 *10' # 最初はcuda fp16i8、後続の10層はcpu fp32
# 'cuda fp16i8 *10+' # 最初は10層cuda int8量子化、残りは必要に応じて動的ロード

# CUDAが使える場合は1、使えない場合は0にする
RWKV_CUDA_ON = "1" #@param [0, 1]

# チャットで使う言語
CHAT_LANG = 'Japanese' #@param ["Japanese", "English", "Chinese"]
# English
# Chinese
# Japanese

# モデルのコンテキストの長さ
ctx_len = 1024 #@param {type:"integer"}

# chat.pyの内容を置換するための辞書を作成
replacements = {
    'args.strategy = .*' : 'args.strategy = \'' + strategy + '\'',
    'os\.environ\[\"RWKV_CUDA_ON\"\] = \'.' : 'os.environ["RWKV_CUDA_ON"] = \'' + str(RWKV_CUDA_ON),
    'CHAT_LANG = .*' : 'CHAT_LANG = \'' + CHAT_LANG + '\'',
    'args.MODEL_NAME = .*' : 'args.MODEL_NAME = \'' + MODEL_NAME + '\'',
    'args.ctx_len = .*' : 'args.ctx_len = \'' + str(ctx_len) + '\'',
    'current_path = os\.path\.dirname\(os\.path\.abspath\(__file__\)\)' : 'current_path = os.getcwd()',
    'while True:\s+msg = prompt.+\s+if len\(msg.+\s+on_message\(msg\)\s+else:\s+print\(.+' : ''
}


In [ ]:
# chat.pyの変数を置換してchat-notebook.pyに保存
import re

with open('chat.py', encoding='utf-8') as f:
    text = f.read()

for old, new in replacements.items():
    pattern = re.compile(r'^(\s*)' + old, flags=re.MULTILINE)
    text = pattern.sub(r'\1' + new, text)

with open('chat-notebook.py', 'w', encoding='utf-8') as f:
    f.write(text)

In [ ]:
# モデルをロード
execfile("chat-notebook.py")



ChatRWKV v2 https://github.com/BlinkDL/ChatRWKV

Japanese - cuda fp16 - /content/ChatRWKV/v2/prompt/default/Japanese-2.py


Using /root/.cache/torch_extensions/py39_cu118 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py39_cu118/wkv_cuda...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py39_cu118/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv_cuda...


Loading model - RWKV-4-Raven-3B-v8-EngAndMore-20230408-ctx4096
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 6

Loading RWKV-4-Raven-3B-v8-EngAndMore-20230408-ctx4096.pth ...
Strategy: (total 32+1=33 layers)
* cuda [float16, float16], store 33 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 25-cuda-float16-float16 26-cuda-float16-float16 27-cuda-float16-float16 28-cuda-float16-float16 29-cuda-float16-float16 30-cuda-float16-float16 31-cuda-float16-fl

# チャット

In [ ]:
# 文章生成に関する設定

# チャットの短い応答の長さ
CHAT_LEN_SHORT = 40 #@param {type:"integer"}
# チャットの長い応答の長さ
CHAT_LEN_LONG = 150 #@param {type:"integer"}
# 自由生成文の長さ
FREE_GEN_LEN = 256 #@param {type:"integer"}

# For better chat & QA quality: reduce temp, reduce top-p, increase repetition penalties
# Explanation: https://platform.openai.com/docs/api-reference/parameter-details

# GEN_TEMPとGEN_TOP_P: 小さい値では正確さが上がり、大きい値では多様性が上がる
GEN_TEMP = 1.1 #@param {type:"number"} # sometimes it's a good idea to increase temp. try it
GEN_TOP_P = 0.7 #@param {type:"number"}
# GEN_alpha_presence, GEN_alpha_frequency: 繰り返し文字列の存在と頻度に対するペナルティ。大きい値では繰り返しが抑制される。
GEN_alpha_presence = 0.2 #@param {type:"number"} # Presence Penalty
GEN_alpha_frequency = 0.2 #@param {type:"number"} # Frequency Penalty
# AVOID_REPEAT: 繰り返しを防ぐ文字
AVOID_REPEAT = '，：？！' #@param {type:"string"}
# 入力を分割するチャンク長
CHUNK_LEN = 256 #@param {type:"integer"} # split input into chunks to save VRAM (shorter -> slower)

AVOID_REPEAT_TOKENS = []
for i in AVOID_REPEAT:
    dd = pipeline.encode(i)
    assert len(dd) == 1
    AVOID_REPEAT_TOKENS += dd


In [ ]:
# チャットの実行
while True:
    msg = input(f'{user}{interface} ')
    if len(msg.strip()) > 0:
        on_message(msg)
    else:
        print('Error: please say something')

Bob: こんにちは
Alice: こんにちは！何かお茶を注文してくれますか？

Bob: 日本の首都はどこですか？
Alice: 日本の首都は東京です。今日は湯川若菜さんにお話を伺いました。日本の首都の中でも、東京は世界で最も有名な都市です。東京には著名な商店街や国立公園があります。

Bob: 旅行は好きですか？
Alice: はい、日本の旅行は素晴らしいですね！都市を探検し、山や海、峡谷を歩くことで、大自然を体験することができます。東京は地域によって、国内外から人気があります。さらに、湯川若菜さんにアドバイスしてみたいと言われるときは、特に何もしなくても良いですね。

